# Vision Encoder

> Wrappers for vision models (e.g., ResNet, ViT) using `timm`, compatible with fast.ai.

In [ ]:
#| default_exp model.vision

## Colab Setup

In [ ]:
#| hide
# Mount Google Drive (Optional, but recommended for persistent storage)
from pathlib import Path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ModuleNotFoundError:
    print("Not running in Colab, skipping Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

Mounted at /content/drive
Google Drive mounted successfully.


In [ ]:
#| export
try:
    import indic_clip.core
    print("Reloaded indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")
            # raise # Stop execution if core components missing

indic_clip.core not found initially.
Added /content/drive/MyDrive/Indic-Clip to sys.path
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive detected, setting PROJECT_ROOT to /content/drive/MyDrive/Indic-Clip
Ensure your project files are located there.
Imported indic_clip.core after path adjustment.


In [ ]:
#| hide
%cd /content/drive/MyDrive/Indic-Clip/

/content/drive/MyDrive/Indic-Clip


In [ ]:
#| hide
!pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torch.nn as nn
import timm
import logging

from fastai.vision.all import *

try:
    from indic_clip.core import get_logger, setup_logging
except ModuleNotFoundError:
    # Fallback if core not found (e.g. testing)
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    def setup_logging(): pass

setup_logging()
logger = get_logger(__name__)

## Vision Encoder Wrapper

In [ ]:
#| export
class VisionEncoder(Module):
    """A wrapper for `timm` vision models to extract features.

    Provides a consistent interface for different vision backbones like
    ResNet, ViT, etc., loading pre-trained weights from `timm`.
    Designed for integration into CLIP-like architectures.
    """
    def __init__(self, model_name: str = 'vit_base_patch16_224',
                 pretrained: bool = True,
                 output_dim: int | None = None):
        """
        Initializes the Vision Encoder.

        Args:
            model_name (str): The name of the `timm` model to load (e.g., 'resnet50', 'vit_base_patch16_224').
            pretrained (bool): Whether to load `timm`'s pre-trained weights (usually ImageNet).
            output_dim (int | None): Optional dimension for a final projection layer.
                                     If None, returns the raw features from the backbone.
                                     If an int, adds a Linear layer to project features to this dimension.
                                     This is typically handled by the main CLIP model's projection head,
                                     so it might be better left as None here.
        """
        self.model_name = model_name
        self.pretrained = pretrained
        self.output_dim = output_dim

        try:
            # Load the model from timm
            # `num_classes=0` removes the final classification layer
            # `global_pool='avg'` ensures avg pooling for CNNs, ViTs might handle pooling differently (e.g., CLS token)
            self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, global_pool='avg')
            logger.info(f"Loaded timm model: {model_name} with pretrained={pretrained}")

            # Get the feature dimension from the backbone
            # For ViTs, this is often backbone.embed_dim
            # For CNNs, it's often backbone.num_features
            if hasattr(self.backbone, 'num_features'):
                self.feature_dim = self.backbone.num_features
            elif hasattr(self.backbone, 'embed_dim'):
                 # Common for ViT models in timm
                 self.feature_dim = self.backbone.embed_dim
            else:
                # Fallback: try forward pass with dummy data (can be slow)
                try:
                     with torch.no_grad():
                         dummy_input = torch.randn(1, 3, 224, 224) # Assuming 224x224 input
                         if '224' not in model_name:
                            logger.warning(f"Model name '{model_name}' doesn't specify input size, assuming 224x224 for feature dim check.")
                         dummy_output = self.backbone(dummy_input)
                         self.feature_dim = dummy_output.shape[-1]
                         logger.info(f"Inferred feature dimension via dummy forward pass: {self.feature_dim}")
                except Exception as e:
                    logger.error(f"Could not automatically determine feature dimension for {model_name}. Error: {e}")
                    self.feature_dim = None
                    raise ValueError(f"Could not determine feature dimension for {model_name}") from e

            logger.info(f"Backbone feature dimension: {self.feature_dim}")

            # Optional projection layer
            self.projection = nn.Identity()
            if output_dim is not None and self.feature_dim is not None:
                self.projection = nn.Linear(self.feature_dim, output_dim)
                logger.info(f"Added projection layer: Linear({self.feature_dim}, {output_dim})")
            elif output_dim is not None and self.feature_dim is None:
                 logger.warning(f"output_dim specified ({output_dim}) but could not determine backbone feature dimension. Projection layer skipped.")

            # Placeholder for gradient checkpointing enabling
            self._gradient_checkpointing = False

        except Exception as e:
            logger.error(f"Failed to initialize VisionEncoder with model {model_name}: {e}")
            raise

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Performs the forward pass to extract image features.

        Args:
            x (torch.Tensor): Input image tensor (batch_size, channels, height, width).

        Returns:
            torch.Tensor: Output features (batch_size, feature_dim or output_dim).
        """
        # Handle gradient checkpointing if enabled and backbone supports it
        if self._gradient_checkpointing and hasattr(self.backbone, 'set_grad_checkpointing'):
             # Timm models often handle this internally, check if this is needed
             # Or apply torch.utils.checkpoint if needed around specific blocks
             # features = torch.utils.checkpoint.checkpoint(self.backbone, x)
             features = self.backbone(x)
        else:
            features = self.backbone(x)

        # Handle potential differences in output format (e.g., ViT CLS token vs avg pool)
        # Timm's create_model with num_classes=0 and global_pool='avg' generally gives [B, D]
        # If using a ViT model specifically, might need features = features[:, 0] if CLS token is used and global_pool='token'
        # Check the specific timm model's behavior if needed.

        # Apply optional projection
        projected_features = self.projection(features)
        return projected_features

    def set_gradient_checkpointing(self, enable: bool = True):
        """Enables or disables gradient checkpointing for the backbone.
           Note: Actual application depends on timm model support or custom checkpointing.
        """
        if hasattr(self.backbone, 'set_grad_checkpointing'):
            self.backbone.set_grad_checkpointing(enable=enable)
            self._gradient_checkpointing = enable # Store state
            logger.info(f"Gradient checkpointing {'enabled' if enable else 'disabled'} via backbone method.")
        else:
            # Store the desired state, forward pass logic might need adjustment
            # if manual torch.utils.checkpoint is used
            self._gradient_checkpointing = enable
            logger.warning(f"Backbone {self.model_name} might not directly support set_grad_checkpointing. Manual checkpointing may be needed in forward pass.")


## Example Usage

In [ ]:
#| eval: false
if __name__ == '__main__':
    print("--- ResNet18 Example ---")
    # Example with ResNet18 and a projection layer
    try:
        resnet_encoder = VisionEncoder(model_name='resnet18', pretrained=True, output_dim=512)
        print(f"Encoder Type: {type(resnet_encoder)}")
        print(f"Feature Dimension: {resnet_encoder.feature_dim}")
        print(f"Output Dimension (after projection): {resnet_encoder.output_dim}")

        # Create a dummy input batch
        dummy_images = torch.randn(4, 3, 224, 224) # Batch size 4
        print(f"Input shape: {dummy_images.shape}")

        # Perform forward pass
        resnet_encoder.eval() # Set to evaluation mode
        with torch.no_grad():
            output_features = resnet_encoder(dummy_images)

        print(f"Output shape: {output_features.shape}")
        print(f"Output dtype: {output_features.dtype}")
        assert output_features.shape == (4, 512)

    except Exception as e:
        print(f"Error during ResNet18 example: {e}")
        import traceback
        traceback.print_exc()

    print("\n--- ViT Example (No Projection) ---")
    # Example with ViT and no projection layer
    try:
        vit_encoder = VisionEncoder(model_name='vit_base_patch16_224', pretrained=True, output_dim=None)
        print(f"Encoder Type: {type(vit_encoder)}")
        print(f"Feature Dimension: {vit_encoder.feature_dim}")
        print(f"Output Dimension (after projection): {vit_encoder.output_dim}")

        # Enable gradient checkpointing (example)
        # vit_encoder.set_gradient_checkpointing(True)
        vit_encoder.set_gradient_checkpointing(False) # Keep it off for simple test

        # Create a dummy input batch
        dummy_images_vit = torch.randn(2, 3, 224, 224) # Batch size 2
        print(f"Input shape: {dummy_images_vit.shape}")

        # Perform forward pass
        vit_encoder.eval()
        with torch.no_grad():
            output_features_vit = vit_encoder(dummy_images_vit)

        print(f"Output shape: {output_features_vit.shape}")
        print(f"Output dtype: {output_features_vit.dtype}")
        assert output_features_vit.shape == (2, 768)

    except Exception as e:
        print(f"Error during ViT example: {e}")
        import traceback
        traceback.print_exc()

2025-04-18 08:34:02 - __main__ - INFO - Loaded timm model: resnet50 with pretrained=True
2025-04-18 08:34:02 - __main__ - INFO - Backbone feature dimension: 2048
2025-04-18 08:34:02 - __main__ - INFO - Added projection layer: Linear(2048, 512)


--- ResNet50 Example ---
Encoder Type: <class '__main__.VisionEncoder'>
Feature Dimension: 2048
Output Dimension (after projection): 512
Input shape: torch.Size([4, 3, 224, 224])
Output shape: torch.Size([4, 512])
Output dtype: torch.float32


2025-04-18 08:34:05 - __main__ - INFO - Loaded timm model: vit_base_patch16_224 with pretrained=True
2025-04-18 08:34:05 - __main__ - INFO - Backbone feature dimension: 768
2025-04-18 08:34:05 - __main__ - INFO - Loaded timm model: vit_base_patch16_224 with pretrained=True
2025-04-18 08:34:05 - __main__ - INFO - Backbone feature dimension: 768
2025-04-18 08:34:05 - __main__ - INFO - Gradient checkpointing disabled via backbone method.



--- ViT Example (No Projection) ---
Encoder Type: <class '__main__.VisionEncoder'>
Feature Dimension: 768
Output Dimension (after projection): None
Input shape: torch.Size([2, 3, 224, 224])
Output shape: torch.Size([2, 768])
Output dtype: torch.float32


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export